#### Root-finding algorithms

In [2]:
import math
from sympy import *

##### Functions

In [3]:
f = [
  lambda x: math.pow(x,10) - 1, 
  lambda x: math.exp(x) - 3 * x, 
  lambda x: math.pow(x,3) - (9 * x) + 5
  ]

##### Auxiliar Function

In [32]:
def print_results(root, iterations, precision, a, b, func, x):
    print(f'Root: {round(root, 4)}')
    print(f'Iterations: {round(iterations, 4)}')
    print(f'a: {round(a, 4)}')
    print(f'b: {round(b, 4)}')
    print(f'Erro: {round(abs(func(x)) - precision, 4)}')

##### Bissection Method

In [47]:
def bissec(a, b, precision, iterations, f):

  if f(a) * f(b) >= 0:
    print('Invalid interval')
    return None

  k = 1
  M = f(a)
  x = (a + b) / 2
  

  while k < iterations:
    print(round(k,4), round(a,4), round(b,4), round(f(x),4), round(M,4))
    if abs(f(x)) < precision:
      break

    if M * f(x) > 0:
      a = x
      M = f(a)
    else:
      b = x

    if abs(f(x)) < precision:
      break

    x = (a + b) / 2

    k += 1
  
  print_results(x, k, precision, a, b, f, x)

 

In [45]:
bissec(0.5, 1, 0.01, 5, h)

1 0.5 1 -1.3281 0.625
2 0.5 0.75 -0.3809 0.625
3 0.5 0.625 0.1155 0.625
4 0.5625 0.625 -0.1344 0.1155
Root: 0.5781
Iterations: 5
a: 0.5625
b: 0.5938
Erro: -0.0001


In [49]:
bissec(0, 1.3, 0.000000001, 5, f)

1 0 1.3 -0.9865 -1.0
2 0.65 1.3 -0.2237 -0.9865
3 0.975 1.3 2.6267 -0.2237
4 0.975 1.1375 0.7285 -0.2237
Root: 1.0156
Iterations: 5
a: 0.975
b: 1.0563
Erro: 0.1677
